Übung: Canary Deployments
-----------------

In dieser Übung führen wir einen Canary Deployment durch.

Dazu wird ein weiteres Deployment mit der neuen (Canary) Version vom BPMN Frontend erstellt. Der Service hat **keinen** Selector für die Version und steuert so beide Versionen an.

Aus Einfachheitsgründen, verwenden wir die Blue/Green YAML Dateien mit einer neuen Service Datei.

Das passiert in einer eigenen Namespace um die Resultate gezielt Darstellen zu können:

In [ ]:
! kubectl create namespace canary

Deployment und Services wurde in einzelne Dateien aufgeteilt. Die Ingress Ressource, aus Einfachheitsgründen weggelassen.

Nur die Labels des Deploymments und natürlich das Image wurde um eine Version erweitert:

    apiVersion: v1
    kind: Service
    metadata:
      name: bpmn-frontend
      labels:
        app: bpmn-frontend
    spec:
      ...
      selector:
        app: bpmn-frontend
        
- - -

    apiVersion: apps/v1
    kind: Deployment
    metadata:
      name: bpmn-frontend-green
      version: 0.2
    ...
    spec:
      containers:
      - name: bpmn-frontend
        image: registry.gitlab.com/mc-b/misegr/misegr/bpmn-frontend:V0.2
        imagePullPolicy: IfNotPresent   
        
- - -

    apiVersion: apps/v1
    kind: Deployment
    metadata:
      name: bpmn-frontend-blue
      version: 1.0
    ...
    spec:
      containers:
      - name: bpmn-frontend
        image: registry.gitlab.com/mc-b/misegr/misegr/bpmn-frontend:V1.0
        imagePullPolicy: IfNotPresent         

Die Anzahl Pods pro Deployment steuern wir Ausnahmsweise mittels `kubectl`. 

In [ ]:
! kubectl apply -f 09-4-Deployment/bpmn-frontend-deployment-green.yaml --namespace canary
! kubectl apply -f 09-4-Deployment/bpmn-frontend-deployment-blue.yaml --namespace canary

! kubectl scale --replicas=4 deployment/bpmn-frontend-green --namespace canary
! kubectl scale --replicas=1 deployment/bpmn-frontend-blue --namespace canary

! kubectl apply -f 09-4-Deployment/bpmn-frontend-service-canary.yaml --namespace canary

Als Ergebnis haben wir zwei Deployments mit fünf Pods. Vier mit der Version 0.2 und einer mit der Version 1.

In [ ]:
! kubectl get all --namespace canary -o wide
! echo "BPMN Frontend: http://$(cat ~/work/server-ip):$(kubectl get service --namespace canary bpmn-frontend -o=jsonpath='{ .spec.ports[0].nodePort }')/frontend/index.html"

- - -

Aufräumen

In [ ]:
! kubectl delete namespace canary